IMPORTS

In [24]:
import numpy as np 
from PIL import Image 
import os 
import cv2 

CUSTOM CLASS

In [25]:
# custom class to store image pairs  
class ImagePair:
    def __init__(self, img, pred):
        self.img = img
        self.pred = pred 

    def __repr__(self):
        return f"ImagePair(img={self.img}, pred={self.pred})" 

# custom class to store list of image pairs from directories 
class ImagePairSet:
    def __init__(self, img_dir, pred_dir):
        self.img_dir = img_dir
        self.pred_dir = pred_dir
        self.img_paths = self.list_images(img_dir) 
        self.pred_paths = self.find_corresponding_preds(pred_dir) 
        self.image_pairs = self.create_image_pairs() 

    def list_images(self, img_dir): 
        # read all images from directory and return list of image paths 
        img_paths = []
        for img in os.listdir(img_dir):
            if img.endswith(".png"):
                img_paths.append(img_dir + img)
        return img_paths 
    
    def find_corresponding_preds(self, pred_dir):
        # find corresponding predictions for each image in img_dir
        pred_paths = []
        for img_path in self.img_paths:
            img_name = img_path.split('/')[-1]
            pred_path = pred_dir + img_name.replace(".png","_prediction.png")
            pred_paths.append(pred_path)
        return pred_paths 
    
    def create_image_pairs(self):
        # create list of image pairs 
        image_pairs = []
        for i in range(len(self.img_paths)):
            img = Image.open(self.img_paths[i])
            pred = Image.open(self.pred_paths[i])
            image_pairs.append(ImagePair(img, pred))
        return image_pairs

READ IMAGES

In [26]:
dir_images = "/home/rp/abhay_ws/marker_detection_failure_recovery/segmentation_model/test_images_real/" 
dir_predictions = "/home/rp/abhay_ws/marker_detection_failure_recovery/segmentation_model/test_images_real/predictions_20250311-235326/predictions/" 

# create ImagePairSet object
image_pair_set = ImagePairSet(dir_images, dir_predictions) 
print(len(image_pair_set.image_pairs)) 



60


TODO: 

* loop through list of image pairs (exit after first loop) 
* fit parallelogram to segmented area 
* initial gueses <- get 4 possible solutions by PnP 
* for each solution, transform marker to pixel space 
* determine best configuration by reprojection loss 
* optimization loop: compute gradient, update pose estimate, compute reprojection loss 


In [70]:
def fit_parallelogram(binary_image):
    binary_image = np.array(binary_image)   
    # Ensure the input image is binary (already thresholded)
    if len(binary_image.shape) == 3:  # In case the image is a color image (unexpected)
        binary_image = cv2.cvtColor(binary_image, cv2.COLOR_BGR2GRAY)
    
    # Find contours from the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    parallelogram_corners = None

    for contour in contours:
        # Check if the contour is large enough to be considered
        area = cv2.contourArea(contour)
        if area < 100:  # Threshold for ignoring small contours (adjust as needed)
            continue

        # Approximate the contour to a polygon (reduce epsilon for higher accuracy)
        epsilon = 0.02 * cv2.arcLength(contour, True)  # Reduce epsilon for more accuracy
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # If the contour has more than 4 points, apply convex hull and approximate again
        if len(approx) > 4:
            hull = cv2.convexHull(contour)
            epsilon = 0.02 * cv2.arcLength(hull, True)  # Reduce epsilon for more accuracy
            approx = cv2.approxPolyDP(hull, epsilon, True)

        # Check if the polygon has 4 sides (quadrilateral)
        if len(approx) == 4:
            parallelogram_corners = approx.reshape(4, 2)
            break  # Stop after finding the first valid parallelogram
    
    return parallelogram_corners

def get_marker_pose(marker_corners_2d, marker_corners_3d, camera_matrix, dist_coeffs=None):
    """
    Estimates the pose of a marker using the PnP algorithm.
    
    Parameters:
    - marker_corners_2d: A list or array of the marker corners in image space (2D coordinates).
    - marker_corners_3d: A list or array of the marker corners in world space (3D coordinates).
    - camera_matrix: Camera intrinsic matrix (3x3) containing focal lengths and optical center.
    - dist_coeffs: (Optional) Camera distortion coefficients, can be None if no distortion is assumed.
    
    Returns:
    - rotation_vector: Rotation vector (3x1) representing the orientation of the marker.
    - translation_vector: Translation vector (3x1) representing the position of the marker.
    - success: Boolean indicating whether the pose was successfully estimated.
    """
    
    # Ensure the inputs are NumPy arrays
    marker_corners_2d = np.array(marker_corners_2d, dtype=np.float32)
    marker_corners_3d = np.array(marker_corners_3d, dtype=np.float32)
    
    # Solve PnP to get the rotation and translation vectors
    success, rotation_vector, translation_vector = cv2.solvePnP(
        marker_corners_3d,  # 3D points in world space
        marker_corners_2d,  # Corresponding 2D points in image space
        camera_matrix,      # Camera matrix (intrinsics)
        dist_coeffs         # Distortion coefficients (can be None if no distortion)
    )
    
    return rotation_vector, translation_vector, success

# loop through image pairs 
for index, image_pair in enumerate(image_pair_set.image_pairs):

    if index == 10: # select specific image 
        img = image_pair.img
        pred = image_pair.pred

        # show selected image pair 
        # img.show()
        # pred.show()

        # fit parallelogram to pred image 
        corners = fit_parallelogram(pred) 
        print(corners)

        # draw parallelogram on pred image
        pred = np.array(pred)
        cv2.polylines(pred, [corners], True, (255, 0, 0), 2)
        pred = Image.fromarray(pred)
        pred.show() 

        # use PnP algorithm to determine pose of parallelogram 
        # Ensure the inputs are NumPy arrays
        marker_corners_2d = np.array(marker_corners_2d, dtype=np.float32)
        marker_corners_3d = np.array(marker_corners_3d, dtype=np.float32)
    
        # Solve PnP to get the rotation and translation vectors
        success, rotation_vector, translation_vector = cv2.solvePnP(
            marker_corners_3d,  # 3D points in world space
            marker_corners_2d,  # Corresponding 2D points in image space
            camera_matrix,      # Camera matrix (intrinsics)
            dist_coeffs         # Distortion coefficients (can be None if no distortion)
        )

        break 


[[ 29 231]
 [ 50 372]
 [350 374]
 [310 207]]
